In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0,1))

In [ ]:
import pickle
with open('data.pkl', 'rb') as f:   # google colab
    corpus = pickle.load(f)

In [ ]:
data=corpus

In [ ]:
data['dim_0'] = data['dim_0'].apply(lambda x: x.reshape(500,1))


In [ ]:
X = data['dim_0']
y = data['labels']
y = y.astype(int)

In [ ]:
data.columns

Index(['dim_0', 'labels'], dtype='object')

In [ ]:
for i in range(data.shape[0]):
    if True in np.isnan(data['dim_0'][i]):
        print(i)

In [ ]:
input_x = []
for array in X:
    input_x.append(array)

"""
X = pd.DataFrame(df.dim_0.tolist())
X = X.to_numpy()
"""

'\nX = pd.DataFrame(df.dim_0.tolist())\nX = X.to_numpy()\n'

In [ ]:
input_x = np.array(input_x)

In [ ]:
input_x=input_x.reshape(4921, 500)

In [ ]:
input_x.shape

(4921, 500)

In [ ]:
#df1=pd.DataFrame
input_x1=[]
for i in range(input_x.shape[0]):
    input_x1.append(pd.Series(input_x[i]))

In [ ]:
input_x1=pd.Series(input_x1)

In [ ]:
input_x1.shape

(4921,)

In [ ]:
# !pip install sktime

In [ ]:
from sklearn.linear_model import RidgeClassifierCV
from sklearn.pipeline import make_pipeline

from sktime.datasets import load_arrow_head  # univariate dataset
from sktime.datasets import load_basic_motions  # multivariate dataset
from sktime.transformations.panel.rocket import Rocket, MiniRocket


In [ ]:
X_train1, y_train1 = load_arrow_head(split="train", return_X_y=True)
import gc
gc.collect()

8602

In [ ]:
# s=X_train1["dim_0"]

In [ ]:
# 


In [ ]:

# X_test1, y_test1 = load_arrow_head(split="test", return_X_y=True)

In [ ]:
type(X_train1.iloc[0])

pandas.core.series.Series

In [ ]:
test_time_series=input_x[0]

In [ ]:
# x_train=x_train.reshape(3936, 500)

In [ ]:
input_x1=pd.DataFrame(input_x1)

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(input_x1,y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
(x_train.shape)

(3936, 1)

In [ ]:
# x_train = pd.DataFrame(x_train)
#x_test = pd.DataFrame(x_test)

In [ ]:
import gc
gc.collect()

176

In [ ]:
rocket = MiniRocket(num_kernels=500)  # by default, ROCKET uses 10,000 kernels
rocket.fit(x_train)
X_train_transform = rocket.transform(x_train)

In [ ]:
print(X_train_transform.shape)
X_train_transform_temp=np.expand_dims(np.array(X_train_transform), axis=2)
X_train_transform_temp.shape


(3936, 420)


(3936, 420, 1)

In [ ]:
def normalization(X_train, X_test):#, X_val
    scaler = MinMaxScaler(feature_range=(0,1))
    X_train = np.reshape(X_train, (-1,X_train_transform_temp.shape[1]))
    #X_val = np.reshape(X_val, (-1,X_train_transform_temp.shape[1]))
    X_test = np.reshape(X_test, (-1,X_train_transform_temp.shape[1]))
    
    X_train = scaler.fit_transform(X_train)
    #X_val = scaler.transform(X_val)
    X_test = scaler.transform(X_test)
    
    return np.reshape(X_train, (-1,X_train_transform_temp.shape[1],1)), np.reshape(X_test, (-1,X_train_transform_temp.shape[1],1))#, np.reshape(X_val, (-1,X_train_transform_temp.shape[1],1))

In [ ]:
X_train_transform_temp,X_test_transform_temp=normalization(X_train_transform_temp,X_train_transform_temp)

In [ ]:
type(X_train_transform_temp)

numpy.ndarray

In [ ]:
# plt.plot(X_train_transform_temp[0][:1000])

In [ ]:
classifier = RidgeClassifierCV(alphas=np.logspace(-3, 3, 10), normalize=True)
classifier.fit(X_train_transform, y_train)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_base.py:145: FutureWarning: 'normalize' was deprecated in version 1.0 and will be removed in 1.2.
If you wish to scale the data, use Pipeline with a StandardScaler in a preprocessing stage. To reproduce the previous behavior:

from sklearn.pipeline import make_pipeline

model = make_pipeline(StandardScaler(with_mean=False), _RidgeGCV())

If you wish to pass a sample_weight parameter, you need to pass it as a fit parameter to each step of the pipeline as follows:

kwargs = {s[0] + '__sample_weight': sample_weight for s in model.steps}
model.fit(X, y, **kwargs)


  FutureWarning,


RidgeClassifierCV(alphas=array([1.00000000e-03, 4.64158883e-03, 2.15443469e-02, 1.00000000e-01,
       4.64158883e-01, 2.15443469e+00, 1.00000000e+01, 4.64158883e+01,
       2.15443469e+02, 1.00000000e+03]),
                  normalize=True)

In [ ]:

X_test_transform = rocket.transform(x_test)

In [ ]:
classifier.score(X_test_transform, y_test)

0.9258883248730965

In [ ]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0
n_classes = len(np.unique(y_train))

In [ ]:
def transformer_encoder(inputs, head_size, num_heads, ff_dim, dropout=0):
    # Normalization and Attention
    #x = layers.LayerNormalization(epsilon=1e-6)(inputs)
    x = layers.MultiHeadAttention(
        key_dim=head_size, num_heads=num_heads, dropout=dropout
    )(inputs,inputs)
    x = layers.Dropout(dropout)(x)
    res = x + inputs

    # Feed Forward Part
    x = layers.LayerNormalization(epsilon=1e-6)(res)
    x = layers.Conv1D(filters=ff_dim, kernel_size=1, activation="relu")(x)
    x = layers.Dropout(dropout)(x)
    x = layers.Conv1D(filters=inputs.shape[-1], kernel_size=1)(x)
    return x + res

In [ ]:
def build_model(
    input_shape,
    head_size,
    num_heads,
    ff_dim,
    num_transformer_blocks,
    mlp_units,
    dropout=0,
    mlp_dropout=0,
):
    inputs = keras.Input(shape=input_shape)
    x = inputs
    for _ in range(num_transformer_blocks):
        x = transformer_encoder(x, head_size, num_heads, ff_dim, dropout)

    x = layers.GlobalAveragePooling1D(data_format="channels_first")(x)
    for dim in mlp_units:
        x = layers.Dense(dim, activation="relu")(x)
        x = layers.Dropout(mlp_dropout)(x)
    outputs = layers.Dense(n_classes, activation="softmax")(x)
    return keras.Model(inputs, outputs)

In [ ]:
X_train_transform_temp.shape[1:]

(420, 1)

In [ ]:
import gc
gc.collect()

1649

In [ ]:
input_shape = X_train_transform_temp.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    X_train_transform_temp,
    y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_4 (InputLayer)           [(None, 420, 1)]     0           []                               
                                                                                                  
 multi_head_attention_12 (Multi  (None, 420, 1)      7169        ['input_4[0][0]',                
 HeadAttention)                                                   'input_4[0][0]']                
                                                                                                  
 dropout_27 (Dropout)           (None, 420, 1)       0           ['multi_head_attention_12[0][0]']
                                                                                                  
 tf.__operators__.add_24 (TFOpL  (None, 420, 1)      0           ['dropout_27[0][0]',       

ValueError: ignored

In [ ]:
model.save("transformer_banana_muffin")

In [ ]:
scaler.fit(data['dim_0'])
data['dim_0'] = scaler.transform(data['dim_0'])

In [ ]:
!zip -r transformer.zip "/kaggle/working/transformer_banana_muffin"

# Normalize Data

In [ ]:
scaler.fit(data['dim_0'][0])

In [ ]:
data['dim_0'] = data['dim_0'].apply(lambda x: scaler.transform(x))

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(input_x,y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
y_train[y_train == -1] = 0
y_test[y_test == -1] = 0
n_classes = len(np.unique(y_train))

In [ ]:
input_shape = x_train.shape[1:]

model = build_model(
    input_shape,
    head_size=256,
    num_heads=4,
    ff_dim=4,
    num_transformer_blocks=4,
    mlp_units=[128],
    mlp_dropout=0.4,
    dropout=0.25,
)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    metrics=["sparse_categorical_accuracy"],
)
model.summary()

callbacks = [keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)]

model.fit(
    x_train,
    y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=64,
    callbacks=callbacks,
)

model.evaluate(x_test, y_test, verbose=1)

In [ ]:
model.save("transformer_banana_muffin_normalized")

In [ ]:
!zip -r transformer_normalized.zip "/kaggle/working/transformer_banana_muffin_normalized"